In [8]:
import pandas as pd
import pymc as pm
import numpy as np

data_path = '/home/jovyan/STA365/World_Population.csv' 
data = pd.read_csv(data_path)

median_population = data['Population (2023)'].median()
data['Population_Above_Median'] = (data['Population (2023)'] > median_population).astype(int)

X = data['Density  (P/Km²)'].values.astype(np.float64)
y = data['Population_Above_Median'].values

with pm.Model() as logistic_model:
    intercept = pm.Normal('Intercept', mu=0, sigma=10)
    beta_density = pm.Normal('Beta_Density', mu=0, sigma=10)
    
    logit_p = intercept + beta_density * X
    y_obs = pm.Bernoulli('y_obs', pm.math.sigmoid(logit_p), observed=y)

    trace = pm.sample(1000, tune=2000, target_accept=0.95, return_inferencedata=False, progressbar=True)


Auto-assigning NUTS sampler...
Initializing NUTS using jitter+adapt_diag...
Multiprocess sampling (4 chains in 4 jobs)
NUTS: [Intercept, Beta_Density]


Sampling 4 chains for 2_000 tune and 1_000 draw iterations (8_000 + 4_000 draws total) took 21 seconds.
The rhat statistic is larger than 1.01 for some parameters. This indicates problems during sampling. See https://arxiv.org/abs/1903.08008 for details
The effective sample size per chain is smaller than 100 for some parameters.  A higher number is needed for reliable rhat and ess computation. See https://arxiv.org/abs/1903.08008 for details
There were 2000 divergences after tuning. Increase `target_accept` or reparameterize.


# Part 2
Bayesian inference regularizes the likelihood through the prior, enriching the analysis with estimates of uncertainty and prior knowledge, in contrast to classical methods that directly optimize a regularized objective function for point estimates.

In [ ]:
import pandas as pd
import pymc as pm
import numpy as np
import arviz as az

data_path = '/home/jovyan/STA365/World_Population.csv' 
data = pd.read_csv(data_path)

median_population = data['Population (2023)'].median()
data['Population_Above_Median'] = (data['Population (2023)'] > median_population).astype(int)

X = data['Density  (P/Km²)'].values.astype(np.float64)
y = data['Population_Above_Median'].values

if X.ndim == 1:
    X = X.reshape(-1, 1)

nu = 29  
w = 1

with pm.Model() as robust_model:
    beta = pm.Normal('beta', mu=0, sigma=10, shape=X.shape[1])
    intercept = pm.Normal('Intercept', mu=0, sigma=10)

    lambda_inv_sq = pm.Gamma('lambda_inv_sq', alpha=nu/2, beta=nu/2, shape=y.shape[0])
    
    mu = pm.math.dot(X, beta) + intercept
    
    y_obs = pm.StudentT('y_obs', nu=nu, mu=mu, lam=lambda_inv_sq, observed=y)
    
    trace = pm.sample(1000, return_inferencedata=True)

az.plot_trace(trace)
az.summary(trace, round_to=2)

lambda_inv_sq_means = np.mean(trace.posterior['lambda_inv_sq'].values, axis=(0, 1))
outlier_threshold = np.percentile(lambda_inv_sq_means, 10)  # Adjust this threshold as needed
outliers = np.where(lambda_inv_sq_means < outlier_threshold)[0]

print("Potential Outliers:", outliers)
